### party_clf_entity training on 1.18m dataset

In [ ]:
cd /content/drive/MyDrive/lab_work/party_all_classifier/code

/content/drive/MyDrive/lab_work/party_all_classifier/code


In [ ]:
import sklearn.model_selection as ms
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [ ]:
d = pd.read_csv('/content/drive/Shareddrives/Delta Lab/Data/facebook_2020/archive/fb_2020_adid_02282022.csv',
                usecols = ['ad_id', 'pd_id', 'page_id','disclaimer', 'ad_combined_text'])

In [ ]:
d.shape

(1182385, 5)

In [ ]:
d.head()

,ad_id,pd_id,page_id,disclaimer,ad_combined_text
0,x2461917247435095,pd-540707072979860-3,540707072979860,MARYOTT FOR CONGRESS,The race in CA-49 is down to the wire. Congres...
1,x2787243378200248,pd-1754372481245454-2,1754372481245454,Friends of Tara Zrinski,💜 Tara Zrinski is supported by Republicans and...
2,x323896925519508,pd-1594170604242427-3,1594170604242427,Hans D Hochheimer,This is a great video which everybody should w...
3,x2798640176905866,pd-1676291615740480-7,1676291615740480,Homeowners and Tenants United,A billionaire Trump donor is spending millions...
4,x426761618289494,pd-456262374764387-3,456262374764387,Voters Not Politicians,Rep. Ryan Berman is blocking legislation that ...


#### Deal with four missing pd ids

In [ ]:
len(d.loc[d['pd_id'].isna()])

249

In [ ]:
d.loc[d['pd_id'].isna()].page_id.unique()

array([103091551376724, 127913103970095, 111718570352005, 100874024777082])

In [ ]:
# Using Pavel's file to fix the missings
fix = pd.read_csv("/content/drive/Shareddrives/Delta Lab/Data/facebook_2020/raw/fix_pdid.csv")

In [ ]:
fix

,page_id,disclaimer,pd_id,op_num
0,100874024777082,John Jack Ducharme for House District 111,1,1908
1,100874024777082,"John ""Jack"" Ducharme for House District 111",2,2601
2,100874024777082,"John \""Jack\"" Ducharme for House District 111",3,4286
3,103091551376724,Friends of Debbie Boothe-Schmidt,1,832
4,103091551376724,Friends of Debbie Boothe-Schmidt\t,2,949
5,103091551376724,Friends of Debbie Boothe-Schmidt\t,3,2601
6,111718570352005,"Yes on Measure A - Homes for San Diegans, Spon...",1,1731
7,111718570352005,"Yes on Measure A - Homes for San Diegans, Spon...",2,1752
8,111718570352005,"Yes on Measure A - Homes for San Diegans, Spon...",3,1896
9,111718570352005,"Yes on Measure A - Homes for San Diegans, Spon...",4,2093


In [ ]:
fix['pdid_fix']="pd-" + fix['page_id'].apply(str) + "-" + fix['pd_id'].apply(str)

In [ ]:
fix = fix[['page_id', 'disclaimer', 'pdid_fix']]

In [ ]:
d = d.merge(fix, on=['page_id', 'disclaimer'], how='left', indicator=True)

In [ ]:
d._merge.value_counts()

left_only     1182235
both              150
right_only          0
Name: _merge, dtype: int64

In [ ]:
d.loc[d['pd_id'].isna(), 'pd_id']=d['pdid_fix']

In [ ]:
# These disclaimers are not in Pavel's file
d.loc[d['pd_id'].isna()].disclaimer.unique()

array(['The ""C"" Club of Houston',
       'Yes on Measure A - Homes for San Diegans, Sponsored by a Coalition of Civic Nonprofits and Housing Providers. Committee major funding from Affirmed Housing Group Inc Eden Housing Inc Metropolitan Area Advisory Committee (MAAC) Funding details at www.sandiego.gov/donors.',
       'John ""Jack"" Ducharme for House District 111'], dtype=object)

In [ ]:
d.loc[d['pd_id'].isna()].disclaimer.unique()[0]

'The ""C"" Club of Houston'

In [ ]:
d.loc[d['pd_id'].isna()].disclaimer.unique()[1]

'Yes on Measure A - Homes for San Diegans, Sponsored by a Coalition of Civic Nonprofits and Housing Providers. Committee major funding from Affirmed Housing Group Inc Eden Housing Inc Metropolitan Area Advisory Committee (MAAC) Funding details at www.sandiego.gov/donors.'

In [ ]:
d.loc[d['pd_id'].isna()].disclaimer.unique()[2]

'John ""Jack"" Ducharme for House District 111'

In [ ]:
# Manually add this pd id
d.loc[(d['page_id']==127913103970095)&(d['disclaimer']=='The ""C"" Club of Houston'), 'pd_id']='pd-127913103970095-2'

In [ ]:
d.loc[(d['page_id']==111718570352005)&(d['disclaimer']=='Yes on Measure A - Homes for San Diegans, Sponsored by a Coalition of Civic Nonprofits and Housing Providers. Committee major funding from Affirmed Housing Group Inc Eden Housing Inc Metropolitan Area Advisory Committee (MAAC) Funding details at www.sandiego.gov/donors.'), 'pd_id']='pd-111718570352005-2'

In [ ]:
d.loc[(d['page_id']==100874024777082)&(d['disclaimer']=='John ""Jack"" Ducharme for House District 111'), 'pd_id']='pd-100874024777082-2'

In [ ]:
d.loc[d['pd_id'].isna()]

,ad_id,pd_id,page_id,disclaimer,ad_combined_text,pdid_fix,_merge


In [ ]:
del d['_merge']

#### Add party all info from wmp entities file

In [ ]:
wmp = pd.read_csv('/content/drive/Shareddrives/Delta Lab/Data/entities_fb_2020/wmp_fb_entities_v051822.csv',
                  usecols=['pd_id','party_all'])

In [ ]:
d = d.merge(wmp, on='pd_id', how='left', indicator=True)

In [ ]:
d['_merge'].value_counts()

both          900809
left_only     281576
right_only         0
Name: _merge, dtype: int64

In [ ]:
del d['_merge']

In [ ]:
d.head()

,ad_id,pd_id,page_id,disclaimer,ad_combined_text,pdid_fix,party_all
0,x2461917247435095,pd-540707072979860-3,540707072979860,MARYOTT FOR CONGRESS,The race in CA-49 is down to the wire. Congres...,NaN,REP
1,x2787243378200248,pd-1754372481245454-2,1754372481245454,Friends of Tara Zrinski,💜 Tara Zrinski is supported by Republicans and...,NaN,NaN
2,x323896925519508,pd-1594170604242427-3,1594170604242427,Hans D Hochheimer,This is a great video which everybody should w...,NaN,REP
3,x2798640176905866,pd-1676291615740480-7,1676291615740480,Homeowners and Tenants United,A billionaire Trump donor is spending millions...,NaN,NaN
4,x426761618289494,pd-456262374764387-3,456262374764387,Voters Not Politicians,Rep. Ryan Berman is blocking legislation that ...,NaN,MISSING


In [ ]:
len(d.pd_id.unique())

10746

In [ ]:
d.party_all.value_counts()

DEM        390855
REP        388584
MISSING    116313
OTHER        5057
Name: party_all, dtype: int64

In [ ]:
# Only keep entities with party all info
d = d.loc[(d['party_all']!="MISSING")&(d['party_all'].notna())]

In [ ]:
d.party_all.value_counts(dropna=True)

DEM      390855
REP      388584
OTHER      5057
Name: party_all, dtype: int64

In [ ]:
len(d.pd_id.unique())

2477

#### Split pd ids

In [ ]:
d_pdid = d.drop_duplicates(subset=['pd_id'], keep='last')
d_pdid = d_pdid[['pd_id', 'party_all']]

In [ ]:
train, test = ms.train_test_split(d_pdid, train_size=0.8, random_state=123)

In [ ]:
print(train.shape)
print(test.shape)

(1981, 2)
(496, 2)


#### Prepare text for training

In [ ]:
# Deduplicate before concatenate ad texts
dd = d.drop_duplicates(subset=['pd_id', 'ad_combined_text'], keep='last')

In [ ]:
d_pdid_txt = dd.groupby(['pd_id'])['ad_combined_text'].apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
d_pdid_txt['text_length'] = d_pdid_txt['ad_combined_text'].apply(len)

In [ ]:
d_pdid_txt.head()

,pd_id,ad_combined_text,text_length
0,pd-1000253160054572-4,"Our movement achieved something amazing, we un...",43277
1,pd-100131284943324-1,I’ve worked alongside Congressman Abraham and ...,6879
2,pd-100140741454415-2,Chris Bish Candidate for California's 6th Cong...,3023
3,pd-100147314821365-1,Texas Oil & Coal Economy Killer! Biden couldn’...,9075
4,pd-100162588547085-1,New South Super PAC Community Change Voters VO...,3479


In [ ]:
d_pdid_txt['text_length'].describe()

count    2.477000e+03
mean     2.020799e+04
std      5.024862e+04
min      5.400000e+01
25%      3.051000e+03
50%      8.182000e+03
75%      1.946500e+04
max      1.019899e+06
Name: text_length, dtype: float64

In [ ]:
d_pdid_txt['text_length'].max()

1019899

In [ ]:
d_pdid_txt.head()

,pd_id,ad_combined_text,text_length
0,pd-1000253160054572-4,"Our movement achieved something amazing, we un...",43277
1,pd-100131284943324-1,I’ve worked alongside Congressman Abraham and ...,6879
2,pd-100140741454415-2,Chris Bish Candidate for California's 6th Cong...,3023
3,pd-100147314821365-1,Texas Oil & Coal Economy Killer! Biden couldn’...,9075
4,pd-100162588547085-1,New South Super PAC Community Change Voters VO...,3479


In [ ]:
train_pdid = train.merge(d_pdid_txt, how ='left', on='pd_id')

In [ ]:
test_pdid = test.merge(d_pdid_txt, how ='left', on='pd_id')

#### MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mnb_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [ ]:
mnb_params = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

In [ ]:
mnb_grid = GridSearchCV(mnb_clf, mnb_params, cv=5, n_jobs=-1)

In [ ]:
mnb_grid.fit(train_pdid['ad_combined_text'], train_pdid['party_all'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [1, 0.1, 0.01],
                         'tfidf__norm': ('l1', 'l2'),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2), (2, 2)]})

In [ ]:
print("Best Params: ", mnb_grid.best_params_)

Best Params:  {'clf__alpha': 0.01, 'tfidf__norm': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (2, 2)}


In [ ]:
print(metrics.classification_report(test_pdid['party_all'], mnb_grid.predict(test_pdid['ad_combined_text']), digits = 3))

              precision    recall  f1-score   support

         DEM      0.862     0.956     0.907       274
       OTHER      1.000     0.091     0.167        22
         REP      0.900     0.855     0.877       200

    accuracy                          0.877       496
   macro avg      0.921     0.634     0.650       496
weighted avg      0.883     0.877     0.862       496



In [ ]:
# Save model to disk
from joblib import dump, load

dump(mnb_grid, '/content/drive/Shareddrives/Delta Lab/Data/facebook_2020_party_all_pdid/party_clf_pdid_mnb.joblib')

['/content/drive/Shareddrives/Delta Lab/Data/facebook_2020_party_all_pdid/party_clf_pdid_mnb.joblib']

#### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_clf = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=500)),
               ])

In [ ]:
lr_params = [{'clf__penalty': ['l2'],
        'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'clf__solver': ['newton-cg']}] 

In [ ]:
lr_grid = GridSearchCV(lr_clf, lr_params, cv=5, n_jobs=-1) 

In [ ]:
lr_grid.fit(train_pdid['ad_combined_text'], train_pdid['party_all'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        LogisticRegression(max_iter=500))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'clf__penalty': ['l2'],
                          'clf__solver': ['newton-cg']}])

In [ ]:
print("Best Params: ", lr_grid.best_params_)

Best Params:  {'clf__C': 100, 'clf__penalty': 'l2', 'clf__solver': 'newton-cg'}


In [ ]:
print(metrics.classification_report(test_pdid['party_all'], lr_grid.predict(test_pdid['ad_combined_text']), digits = 3))

              precision    recall  f1-score   support

         DEM      0.875     0.916     0.895       274
       OTHER      1.000     0.318     0.483        22
         REP      0.847     0.855     0.851       200

    accuracy                          0.865       496
   macro avg      0.907     0.696     0.743       496
weighted avg      0.869     0.865     0.859       496



In [ ]:
dump(lr_grid, '/content/drive/Shareddrives/Delta Lab/Data/facebook_2020_party_all_pdid/party_clf_pdid_logit.joblib')

['/content/drive/Shareddrives/Delta Lab/Data/facebook_2020_party_all_pdid/party_clf_pdid_logit.joblib']

#### SVM

In [ ]:
from sklearn.svm import SVC  

In [ ]:
from sklearn.linear_model import SGDClassifier
svm_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())])

In [ ]:
svm_params = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

In [ ]:
svm_grid = GridSearchCV(svm_clf, svm_params, cv=5, n_jobs=-1)

In [ ]:
svm_grid.fit(train_pdid['ad_combined_text'], train_pdid['party_all'])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'clf__alpha': (0.01, 0.001),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2)]})

In [ ]:
print("Best Params: ", svm_grid.best_params_)

Best Params:  {'clf__alpha': 0.001, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


In [ ]:
print(metrics.classification_report(test_pdid['party_all'], svm_grid.predict(test_pdid['ad_combined_text']), digits = 3))

              precision    recall  f1-score   support

         DEM      0.832     0.938     0.882       274
       OTHER      1.000     0.045     0.087        22
         REP      0.866     0.805     0.834       200

    accuracy                          0.845       496
   macro avg      0.899     0.596     0.601       496
weighted avg      0.853     0.845     0.827       496



In [ ]:
# save the model to disk
dump(svm_grid, '/content/drive/Shareddrives/Delta Lab/Data/facebook_2020_party_all_pdid/party_clf_pdid_svm.joblib')

['/content/drive/Shareddrives/Delta Lab/Data/facebook_2020_party_all_pdid/party_clf_pdid_svm.joblib']